In [8]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
import h5py

app = Flask(__name__)

# 加载模型
file = h5py.File('model.h5', 'r')
model = keras.models.load_model(file)
file.close()

# 对分类特征进行编码的函数
def encode_features(data):
    label_encoders = {}
    categorical_features = ['Study_Regularity', 'Study_Method', 'Student_Background', 'Study_Start']
    for feature in categorical_features:
        le = LabelEncoder()
        data[feature] = le.fit_transform(data[feature])
        label_encoders[feature] = le
    return data, label_encoders

# 数据预处理和预测的函数
def predict_score(data):
    data, _ = encode_features(data)
    scaler = StandardScaler()
    data[['Study_Hours', 'Exam_Score']] = scaler.fit_transform(data[['Study_Hours', 'Exam_Score']])
    predictions = model.predict(data)
    all_predictions = [prediction[0] + 10 for prediction in predictions]
    average_prediction = np.mean(all_predictions)
    return int(average_prediction)



In [9]:
# 定义API路由
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # 从请求中获取JSON数据
        json_data = request.get_json()
        test_data = pd.DataFrame(json_data)
        
        # 预测分数
        average_prediction = predict_score(test_data)
        # 返回预测结果
        response = {
            'average_prediction': average_prediction
        }
        return jsonify(response)
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 108ms/step


127.0.0.1 - - [26/Oct/2023 09:49:41] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 34ms/step


127.0.0.1 - - [26/Oct/2023 09:49:53] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [26/Oct/2023 09:49:58] "POST /predict HTTP/1.1" 200 -
